## Setup
1. In `cd NEMO`
1. Run YAP API server `./yap api` (if you specify a port, change it in `config.py`)
1. `pip install -r requirements.txt`
1. install bclm https://github.com/OnlpLab/bclm
1. Run NEMO API server `uvicorn api_main:app --reload --port 8090`

In [59]:
import requests

In [60]:
requests.get('http://localhost:8090/').json()

{'error': 'Please specify command'}

In [61]:
texts = [
    'גנן גידל דגן בגן\nדגן גדול גדל בגן',
    'אובאמה הכריז אתמול בעצרת האו"ם שטראמפ ליצן.',
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.'
]

## Run NER Model

In [62]:
%%time 
payload = { 'sentences': texts[1],
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/run_ner_model', params=payload).json()
list(zip(res['tokenized_text'][0], res['nemo_predictions'][0]))

CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 99.7 ms


[('אובאמה', 'S-PER'),
 ('הכריז', 'O'),
 ('אתמול', 'O'),
 ('בעצרת', 'O^O'),
 ('האו"ם', 'B-ORG^E-ORG'),
 ('שטראמפ', 'O^O'),
 ('ליצן', 'O'),
 ('.', 'O')]

### multi_to_single (get token-single predictions using a token-multi model)

In [63]:
%%time
payload = { 'sentences': texts[1],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_to_single', params=payload).json()
res

CPU times: user 10.2 ms, sys: 0 ns, total: 10.2 ms
Wall time: 81.6 ms


{'tokenized_text': [['אובאמה',
   'הכריז',
   'אתמול',
   'בעצרת',
   'האו"ם',
   'שטראמפ',
   'ליצן',
   '.']],
 'nemo_multi_predictions': [['S-PER',
   'O',
   'O',
   'O^O',
   'B-ORG^E-ORG',
   'O^O',
   'O',
   'O']],
 'single_predictions': [['S-PER', 'O', 'O', 'O', 'S-ORG', 'O', 'O', 'O']]}

## MD + NER

### multi_align_hybrid

In [64]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_align_hybrid', params=payload).json()
res

CPU times: user 9.13 ms, sys: 0 ns, total: 9.13 ms
Wall time: 186 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש

In [65]:
print(res['md_lattice'])

0	1	מלך	מלך	NNT	NNT	gen=M|num=S	1
1	2	ה	ה	DEF	DEF	_	2
2	3	שערים	שער	NN	NN	gen=M|num=P	2
3	4	ב	ב	PREPOSITION	PREPOSITION	_	3
4	5	מונדיאל	מונדיאל	NNT	NNT	gen=M|num=S	3
5	6	2006	_	CD	CD	_	4
6	7	היה	היה	COP	COP	gen=M|num=S|per=3	5
7	8	מירוסלב	מירוסלב	NNP	NNP	gen=M|num=S	6
8	9	קלוזה	קלוזה	NNP	NNP	gen=F|gen=M|num=S	7
9	10	.	_	yyDOT	yyDOT	_	8




### morph_yap

In [66]:
%%time
payload = { 'sentences': texts[2],
            #'model_name': 'morph',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/morph_yap', params=payload).json()
res

CPU times: user 8.41 ms, sys: 1.83 ms, total: 10.2 ms
Wall time: 184 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tge

### morph_hybrid
The following models got the best results in our experiments.


In [67]:
%%time
payload = { 'sentences': texts[2],
            #'multi_model_name': 'token-multi',
            #'morph_model_name': 'morph',
            #'tokenized': False,
            # 'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()
res

CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 201 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש

### morph_hybrid_align_tokens

In [68]:
%%time
payload = { 'sentences': texts[2],
            #'tokenized': False,
            #'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid_align_tokens', params=payload).json()
res

CPU times: user 10.3 ms, sys: 0 ns, total: 10.3 ms
Wall time: 232 ms


{'tokenized_text': [['מלך',
   'השערים',
   'במונדיאל',
   '2006',
   'היה',
   'מירוסלב',
   'קלוזה',
   '.']],
 'nemo_multi_predictions': [['O',
   'O^O',
   'O^O',
   'O',
   'O',
   'B-PER',
   'E-PER',
   'O']],
 'ma_lattice': '0\t1\tמל\tמל\tBN\tBN\tgen=M|num=S|per=A\t1\n0\t3\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t1\n0\t5\tמלך\tמלך\tVB\tVB\tgen=M|num=S|per=3|tense=PAST\t1\n0\t5\tמלך\tמלך\tNN\tNN\tgen=M|num=S\t1\n0\t5\tמלך\tמלך\tNNT\tNNT\tgen=M|num=S\t1\n1\t2\tאת\tאת\tPOS\tPOS\t_\t1\n2\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n2\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n3\t4\tל\tל\tIN\tIN\t_\t1\n3\t5\tלך\tהלך\tVB\tVB\tgen=M|num=S|per=2|tense=IMPERATIVE\t1\n4\t5\tאת\tאת\tS_PRN\tS_PRN\tgen=F|num=S|per=2\t1\n4\t5\tאתה\tאתה\tS_PRN\tS_PRN\tgen=M|num=S|per=2\t1\n5\t6\tה\tה\tDEF\tDEF\t_\t2\n5\t7\tה\tה\tREL\tREL\t_\t2\n5\t8\tהשערים\tהשערים\tNNP\tNNP\tgen=F|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=S\t2\n5\t8\tהשערים\tהשערים\tNN\tNN\tgen=M|num=P|num=S\t2\n5\t8\tהשערים\tהש

## Working with sequence labels
`iobes` can be used to parse the predictions (`pip install iobes`)

In [69]:
import iobes
import pandas as pd

In [70]:
payload = { 'sentences': '\n'.join(texts),}

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()

In [71]:
def get_ents(toks, labels):
    ents = []
    for i, (sl, sm) in enumerate(zip(labels, toks)):
        spans = iobes.parse_spans_iobes(sl)
        for span in spans:
            text = ' '.join(sm[span.start:span.end])
            ents.append({ 'sent_id': i, 
                          'text': text, 
                          'cat': span.type})
    return pd.DataFrame(ents)

In [72]:
labels = res['nemo_morph_predictions']
morphs = res['morph_segmented_text']
ents_hyb = get_ents(morphs, labels)
ents_hyb

Illegal Label: `B` as final token 4
Illegal Label: `I` as final token at 5


,sent_id,text,cat
0,0,גן,LOC
1,1,ה גן,LOC
2,2,אובאמה,PER
3,2,"ה או""ם",ORG
4,3,מירוסלב קלוזה,PER


In [73]:
labels = res['morph_aligned_multi_predictions']
morphs = res['morph_segmented_text']
ents_align = get_ents(morphs, labels)
ents_align

,sent_id,text,cat
0,2,אובאמה,PER
1,2,"ה או""ם",ORG
2,3,מירוסלב קלוזה,PER
